In [ ]:
# 開発過程の補足

# この Jupyter Notebook ファイルは、開発過程を示すために作成されています。  
# セル単位でコードを実行しながら処理内容を確認する「検証用環境」として使用しています。  
# 実際の本番環境では VSCode 上でモジュールを作成し、AWS Lambda および Step Functions によりデータ処理を実行しています。

In [ ]:
# セル1: 必要なライブラリのインポート
import pandas as pd
import numpy as np
import os
import yaml
import json
import logging
from datetime import datetime
from pathlib import Path

In [ ]:
# セル2: 設定とユーティリティ関数
class ProductCalculator:
    """
    商品データに対して計算処理を行うクラス
    
    統合されたCSVデータを読み込み、追加の計算・分析を行って
    新しい列を追加し、結果を保存します。
    """
    
    def __init__(self, config_path=None):
        """
        ProductCalculatorの初期化
        
        Parameters:
        -----------
        config_path : str, optional
            設定ファイルのパス（指定しない場合はデフォルトパスを使用）
        """
        # プロジェクトルートディレクトリの検出
        self.root_dir = self._find_project_root()
        
        # ディレクトリパスの設定
        self.data_dir = os.path.join(self.root_dir, 'data')
        self.log_dir = os.path.join(self.root_dir, 'logs')
        
        # ディレクトリが存在しない場合は作成
        os.makedirs(self.data_dir, exist_ok=True)
        os.makedirs(self.log_dir, exist_ok=True)
        
        # ログの設定
        self.setup_logging()
        
        # 設定ファイルの読み込み
        self.config = self._load_config(config_path)
        
        # 入出力ファイルパスの設定
        self.setup_file_paths()
    
    def _find_project_root(self):
        """
        プロジェクトのルートディレクトリを検出する
        
        Returns:
        --------
        str
            プロジェクトルートディレクトリのパス
        """
        # 現在のファイルの絶対パスを取得
        current_dir = os.path.abspath(os.getcwd())
        
        # 親ディレクトリを探索
        path = Path(current_dir)
        while True:
            # .gitディレクトリがあればそれをルートとみなす
            if (path / '.git').exists():
                return str(path)
            
            # プロジェクトのルートを示す他のファイル/ディレクトリの存在チェック
            if (path / 'README.md').exists():
                return str(path)
            
            # これ以上上の階層がない場合は現在のディレクトリを返す
            if path.parent == path:
                return str(path)
            
            # 親ディレクトリへ
            path = path.parent
    
    def setup_logging(self):
        """ログ機能のセットアップ"""
        # すべての既存のログハンドラを削除
        root_logger = logging.getLogger()
        for handler in root_logger.handlers[:]:
            root_logger.removeHandler(handler)
        
        # ログファイルパスの設定
        log_filename = f'product_calculator_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'
        log_file = os.path.join(self.log_dir, log_filename)
        
        # 基本設定 - filehandlerとconsolehandlerを使って明示的に設定
        # logging.basicConfigではなく、個別にハンドラを追加
        root_logger.setLevel(logging.INFO)
        
        # ファイルハンドラ
        file_handler = logging.FileHandler(log_file, encoding='utf-8')
        file_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(file_formatter)
        root_logger.addHandler(file_handler)
        
        # コンソールハンドラ
        console_handler = logging.StreamHandler()
        console_handler.setLevel(logging.INFO)
        console_formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(console_formatter)
        root_logger.addHandler(console_handler)
        
        # ログファイルの場所を明示的に表示 - 通常のprintで
        print(f"ログファイル出力先: {log_file}")
        # ここで一度だけログ記録
        logging.info(f"ログ機能の初期化が完了しました: {log_file}")
    
    def _load_config(self, config_path=None):
        """
        設定ファイルを読み込む
        
        Parameters:
        -----------
        config_path : str, optional
            設定ファイルのパス
            
        Returns:
        --------
        dict
            設定データ
        """
        # デフォルトのパス
        if config_path is None:
            config_path = os.path.join(self.root_dir, 'config', 'settings.yaml')
        
        print(f"設定ファイルパス: {config_path}")
        
        try:
            if not os.path.exists(config_path):
                raise FileNotFoundError(f"設定ファイルが見つかりません: {config_path}")
                
            # YAMLファイルを読み込む
            with open(config_path, 'r', encoding='utf-8') as f:
                config = yaml.safe_load(f)
            
            # 計算機能の設定確認
            if 'calculator' not in config:
                config['calculator'] = {}
            
            # デフォルトの出力設定（なければ設定）
            if 'output' not in config['calculator']:
                config['calculator']['output'] = {
                    'input_file': 'integrated_data.csv',
                    'output_file': 'calculated_data.csv'
                }
            
            print("設定ファイルを正常に読み込みました")
            return config
        except Exception as e:
            logging.error(f"設定ファイルの読み込みエラー: {str(e)}")
            raise
    
    def setup_file_paths(self):
        """入出力ファイルパスの設定"""
        # 設定から入出力ファイル名を取得
        input_filename = self.config['calculator']['output'].get('input_file', 'integrated_data.csv')
        output_filename = self.config['calculator']['output'].get('output_file', 'calculated_data.csv')
        
        # 相対パスを絶対パスに変換
        if not os.path.isabs(input_filename):
            self.input_file = os.path.join(self.data_dir, input_filename)
        else:
            self.input_file = input_filename
            
        if not os.path.isabs(output_filename):
            self.output_file = os.path.join(self.data_dir, output_filename)
        else:
            self.output_file = output_filename
        
        logging.info(f"入力ファイル: {self.input_file}")
        logging.info(f"出力ファイル: {self.output_file}")

In [ ]:
# セル3: データ読み込みと基本計算機能
class ProductCalculator(ProductCalculator):
    def load_data(self):
        """
        CSVデータを読み込む
        
        Returns:
        --------
        pandas.DataFrame
            読み込んだデータフレーム
        """
        try:
            # ファイルの存在確認
            if not os.path.exists(self.input_file):
                raise FileNotFoundError(f"入力ファイルが見つかりません: {self.input_file}")
            
            # CSVファイルの読み込み
            df = pd.read_csv(self.input_file, encoding='utf-8-sig')
            
            logging.info(f"データを読み込みました: {len(df)}行, {len(df.columns)}列")
            print(f"📊 {len(df)}行のデータを読み込みました")
            
            return df
        except Exception as e:
            logging.error(f"データ読み込みエラー: {str(e)}")
            raise
    
    def save_data(self, df):
        """
        計算結果をCSVとして保存
        
        Parameters:
        -----------
        df : pandas.DataFrame
            保存するデータフレーム
        """
        try:
            # 出力ディレクトリの確認
            output_dir = os.path.dirname(self.output_file)
            if output_dir and not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # 元のデータフレームをコピー
            output_df = df.copy()
            
            # 不要な列を除外
            # 「ネッシー_」「スーデリ_」「ヤフー_」「ヨリヤス_」で始まる列を除外
            columns_to_drop = [col for col in output_df.columns if 
                             col.startswith('ネッシー_') or 
                             col.startswith('スーデリ_') or 
                             col.startswith('ヤフー_') or 
                             col.startswith('ヨリヤス_')]
            
            if columns_to_drop:
                # 不要な列を削除
                output_df = output_df.drop(columns=columns_to_drop)
                logging.info(f"{len(columns_to_drop)}列の仕入れソースデータを除外しました")
                print(f"ℹ️ {len(columns_to_drop)}列の仕入れソースデータを除外しました")
            
            # CSVとして保存
            output_df.to_csv(self.output_file, index=False, encoding='utf-8-sig')
            
            logging.info(f"データを保存しました: {self.output_file} ({len(output_df)}行, {len(output_df.columns)}列)")
            print(f"✅ {len(output_df)}行のデータを {self.output_file} に保存しました")
        except Exception as e:
            logging.error(f"データ保存エラー: {str(e)}")
            raise
    
    def add_calculation_columns(self, df):
        """
        計算列を追加する
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成（元データを変更しないため）
            result_df = df.copy()
            
            # ここに計算処理を実装していきます
            # === 基本的な計算処理の例 ===

            # 販売価格の合計計算
            # カート販売価格の合計
            if 'カート価格' in result_df.columns:
                result_df['販売価格_カート合計'] = result_df['カート価格'].fillna(0) + result_df['カート価格送料'].fillna(0) + result_df['カート価格のポイント'].fillna(0)
            
            # FBA販売価格の合計
            if 'FBA最安値' in result_df.columns:
                result_df['販売価格_FBA合計'] = result_df['FBA最安値'].fillna(0) + result_df['FBA最安値のポイント'].fillna(0)
            
            # 自己発送販売価格の合計
            if '自己発送最安値' in result_df.columns:
                result_df['販売価格_自己発合計'] = result_df['自己発送最安値'].fillna(0) + result_df['自己発送最安値の送料'].fillna(0) + result_df['自己発送最安値のポイント'].fillna(0)

            # 最小販売価格（カート・FBA・自己発送のうち、0より大きい値の最小値）
            if 'カート価格' in result_df.columns:
                result_df['販売価格_設定販売額'] = result_df[['販売価格_カート合計', '販売価格_FBA合計', '販売価格_自己発合計']].replace(0, np.nan).min(axis=1)
            
            # サイズの計算
            # サイズ_合計cm（三辺の合計）
            if 'パッケージ最長辺' in result_df.columns:
                result_df['サイズ_合計cm'] = result_df['パッケージ最長辺'].fillna(0) + result_df['パッケージ中辺'].fillna(0) + result_df['パッケージ最短辺'].fillna(0)
            
            # サイズ_合計cm3（体積）
            if 'パッケージ最長辺' in result_df.columns:
                result_df['サイズ_合計cm3'] = result_df['パッケージ最長辺'].fillna(0) * result_df['パッケージ中辺'].fillna(0) * result_df['パッケージ最短辺'].fillna(0)

            # サイズ_小型標準判定（小型標準サイズの判定）
            if 'パッケージ最長辺' in result_df.columns and 'パッケージ重量' in result_df.columns:
                result_df['サイズ_小型標準判定'] = np.where((result_df['パッケージ最長辺'].fillna(0) <= 25) & (result_df['パッケージ中辺'].fillna(0) <= 18) & (result_df['パッケージ最短辺'].fillna(0) <= 2) & (result_df['パッケージ重量'].fillna(0) <= 250), '対象', '対象外')


            # 出品者_amazon（Amazonが出品しているかどうかの判定）
            if 'Amazon価格' in result_df.columns:
                result_df['出品者_amazon'] = np.where(result_df['Amazon価格'].fillna(0) >= 1, '有', '無')


                        
            logging.info(f"計算処理が完了しました: {len(result_df.columns) - len(df.columns)}列追加")
            return result_df
            
        except Exception as e:
            logging.error(f"計算処理エラー: {str(e)}")

In [ ]:
# セル4: JSONデータを使用した高度な計算機能
class ProductCalculator(ProductCalculator):
    def load_json_data(self, json_file_path):
        """
        JSONファイルからデータを読み込む
        
        Parameters:
        -----------
        json_file_path : str
            JSONファイルのパス
            
        Returns:
        --------
        dict
            JSONデータ
        """
        try:
            if not os.path.exists(json_file_path):
                logging.warning(f"JSONファイルが見つかりません: {json_file_path}")
                return {}
            
            with open(json_file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            logging.info(f"JSONデータを読み込みました: {json_file_path}")
            return data
        except Exception as e:
            logging.error(f"JSONデータの読み込みエラー: {str(e)}")
            return {}
    
    def add_json_based_calculations(self, df, json_data):
        """
        JSON参照データに基づく計算を追加
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
        json_data : dict
            参照用JSONデータ
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # JSONデータが空の場合は処理しない
            if not json_data:
                logging.warning("JSONデータが空です。JSON参照による計算はスキップします。")
                return result_df
            
            # 例: JSONデータを使用したカテゴリコード→カテゴリ名の変換
            if 'カテゴリー' in result_df.columns and 'categories' in json_data:
                # カテゴリコードをキー、カテゴリ名を値とした辞書を作成
                category_map = {str(item['id']): item['name'] for item in json_data.get('categories', [])}
                
                # マッピングを適用
                result_df['カテゴリ名'] = result_df['カテゴリー'].astype(str).map(category_map)
                logging.info("'カテゴリ名'列を追加しました")
                
            # 例: 商品の季節性スコア計算（JSONからの情報に基づく）
            if 'ASIN' in result_df.columns and 'seasonal_scores' in json_data:
                # ASINをキーとした季節性スコアの辞書を作成
                seasonal_map = json_data.get('seasonal_scores', {})
                
                # マッピングを適用
                result_df['季節性スコア'] = result_df['ASIN'].map(seasonal_map)
                
                # 見つからない場合のデフォルト値を設定
                result_df['季節性スコア'] = result_df['季節性スコア'].fillna(0)
                logging.info("'季節性スコア'列を追加しました")
            
            # 例: 商品の複雑なカテゴリ分類（JSONからの階層情報に基づく）
            if 'カテゴリー' in result_df.columns and 'category_hierarchy' in json_data:
                category_hierarchy = json_data.get('category_hierarchy', {})
                
                # カテゴリ階層情報を取得する関数
                def get_category_hierarchy(category_id):
                    category_id = str(category_id)
                    if category_id in category_hierarchy:
                        return category_hierarchy[category_id]
                    return []
                
                # 階層情報をリストとして追加
                result_df['カテゴリ階層'] = result_df['カテゴリー'].astype(str).apply(get_category_hierarchy)
                
                # 必要に応じて階層情報から特定の階層を抽出
                result_df['カテゴリL1'] = result_df['カテゴリ階層'].apply(lambda x: x[0] if len(x) > 0 else "")
                result_df['カテゴリL2'] = result_df['カテゴリ階層'].apply(lambda x: x[1] if len(x) > 1 else "")
                
                # リスト形式のカラムは使いにくいので削除
                result_df = result_df.drop('カテゴリ階層', axis=1)
                
                logging.info("カテゴリ階層情報の列を追加しました")
            
            logging.info(f"JSON参照による計算処理が完了しました")
            return result_df
            
        except Exception as e:
            logging.error(f"JSON参照による計算処理エラー: {str(e)}")
            # エラーが発生してもできるだけ処理を続行
            return df

    def add_size_calculations(self, df):
        """
        サイズに関する計算を行うメソッド
        JSONファイルからサイズ区分データを読み込み、サイズ判定を行います
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # JSONファイルからサイズデータを読み込む
            json_file_path = os.path.join(self.root_dir, 'config', 'shipping_size_data.json')
            if not os.path.exists(json_file_path):
                logging.warning(f"サイズデータファイルが見つかりません: {json_file_path}")
                return result_df
            
            with open(json_file_path, 'r', encoding='utf-8') as f:
                size_data = json.load(f)
            
            # サイズ区分データを取得
            size_categories = size_data.get('サイズ区分', {})
            
            # 在庫保管手数料データを取得
            storage_fees = size_data.get('在庫保管手数料', {})
            
            # サイズ判定関数
            def determine_size_category(row):
                # サイズと重量情報を取得
                sum_of_edges = row['サイズ_合計cm'] if pd.notna(row['サイズ_合計cm']) else 0
                longest_edge = row['パッケージ最長辺'] if pd.notna(row['パッケージ最長辺']) else 0
                middle_edge = row['パッケージ中辺'] if pd.notna(row['パッケージ中辺']) else 0
                shortest_edge = row['パッケージ最短辺'] if pd.notna(row['パッケージ最短辺']) else 0
                weight = row['パッケージ重量'] if pd.notna(row['パッケージ重量']) else 0
                
                # サイズ区分上限を取得
                size_limits = size_data.get('サイズ区分上限', {})
                
                # 主要カテゴリの判定（小型から順に判定）
                if (weight <= size_limits['小型']['最大重量'] and 
                    sum_of_edges <= size_limits['小型']['最大寸法']['三辺合計'] and
                    longest_edge <= size_limits['小型']['最大寸法']['最長辺'] and
                    middle_edge <= size_limits['小型']['最大寸法']['中辺'] and
                    shortest_edge <= size_limits['小型']['最大寸法']['最短辺']):
                    main_category = "小型"
                elif (weight <= size_limits['標準']['最大重量'] and 
                      sum_of_edges <= size_limits['標準']['最大寸法']['三辺合計'] and
                      longest_edge <= size_limits['標準']['最大寸法']['最長辺'] and
                      middle_edge <= size_limits['標準']['最大寸法']['中辺'] and
                      shortest_edge <= size_limits['標準']['最大寸法']['最短辺']):
                    main_category = "標準"
                elif (weight <= size_limits['大型']['最大重量'] and 
                      sum_of_edges <= size_limits['大型']['最大寸法']['三辺合計']):
                    main_category = "大型"
                elif (weight <= size_limits['特大型']['最大重量'] and 
                      sum_of_edges <= size_limits['特大型']['最大寸法']['三辺合計']):
                    main_category = "特大型"
                else:
                    return "対象外"
                
                # 詳細サイズ区分の判定
                matching_categories = [name for name, data in size_categories.items() 
                                      if name.startswith(main_category) and
                                      weight <= data.get('重量', float('inf')) and
                                      ((('最長辺' in data.get('寸法', {}) and
                                         longest_edge <= data['寸法']['最長辺'] and
                                         middle_edge <= data['寸法'].get('中辺', float('inf')) and
                                         shortest_edge <= data['寸法'].get('最短辺', float('inf')))) or
                                       (('三辺合計' in data.get('寸法', {}) and
                                         sum_of_edges <= data['寸法']['三辺合計'])))]
                
                # 該当するカテゴリが見つかった場合は最初のものを返す
                return matching_categories[0] if matching_categories else main_category
            
            # 月額保管料を計算する関数
            def calculate_storage_fee(row):
                # 体積情報を取得
                volume_cm3 = row['サイズ_合計cm3'] if pd.notna(row['サイズ_合計cm3']) else 0
                
                # サイズカテゴリを取得
                size_category = row['サイズ_大きさ'] if pd.notna(row['サイズ_大きさ']) else "対象外"
                
                # サイズカテゴリが対象外または未定義の場合
                if size_category == "対象外":
                    return None
                
                # メインカテゴリ（小型、標準、大型、特大型）を抽出
                main_category = size_category.split('-')[0] if '-' in size_category else size_category
                
                # 該当するカテゴリの保管料単価を取得
                if main_category in storage_fees:
                    fee_rate = storage_fees[main_category].get('単価', 0)
                    
                    # 1000cm3あたりの料金で計算
                    storage_fee = fee_rate * (volume_cm3 / 1000)
                    
                    # 小数点以下を四捨五入して整数に
                    return round(storage_fee)
                
                return None
            
            # サイズ区分を判定して列に追加
            result_df['サイズ_大きさ'] = result_df.apply(determine_size_category, axis=1)
            
            # 月額保管料を計算して列に追加
            result_df['手数料・利益_月額保管料'] = result_df.apply(calculate_storage_fee, axis=1)
            
            # 配送代行手数料の計算
            if 'サイズ_大きさ' in result_df.columns and '販売価格_設定販売額' in result_df.columns:
                # 手数料を計算する関数
                def calculate_shipping_fee(row):
                    size_category = row['サイズ_大きさ']
                    price = row['販売価格_設定販売額'] if pd.notna(row['販売価格_設定販売額']) else 0
                    
                    # サイズカテゴリが対象外または存在しない場合
                    if size_category == "対象外" or size_category not in size_categories:
                        return None
                    
                    # 価格に応じた手数料を取得
                    fee_data = size_categories[size_category].get('配送代行手数料', {})
                    if price <= 1000:
                        return fee_data.get('1000円以下', None)
                    else:
                        return fee_data.get('1000円超', None)
                
                # 配送代行手数料を計算して列に追加
                result_df['手数料・利益_発送代行手数料'] = result_df.apply(calculate_shipping_fee, axis=1)
            
            return result_df
            
        except Exception as e:
            logging.error(f"サイズ計算処理エラー: {str(e)}")
            logging.error(traceback.format_exc())  # 詳細なエラー情報を出力
            return df


    def add_category_calculations(self, df):
        """
        カテゴリに関する計算を行うメソッド
        JSONファイルからカテゴリデータを読み込み、カテゴリ情報と販売手数料率を追加します
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # JSONファイルからカテゴリデータを読み込む
            json_file_path = os.path.join(self.root_dir, 'config', 'category_data.json')
            if not os.path.exists(json_file_path):
                logging.warning(f"カテゴリデータファイルが見つかりません: {json_file_path}")
                return result_df
            
            with open(json_file_path, 'r', encoding='utf-8') as f:
                category_data = json.load(f)
            
            # カテゴリマッピングデータを取得
            category_mapping = category_data.get('カテゴリマッピング', {})
            
            # カテゴリIDからカテゴリ名へのマッピング作成（逆引き用）
            category_id_to_name = {}
            for category_name, info in category_mapping.items():
                category_id = info.get('keepaカテゴリID')
                if category_id:
                    category_id_to_name[category_id] = category_name
            
            # カテゴリ情報とカテゴリ名から販売手数料率を計算する関数
            def get_category_info_and_fee_rate(row):
                # カテゴリIDを取得
                category_id = str(row['カテゴリー']) if pd.notna(row['カテゴリー']) else ''
                
                # カテゴリIDからカテゴリ名を取得
                category_name = category_id_to_name.get(category_id, '不明')
                
                # 販売価格を取得
                price = row['販売価格_設定販売額'] if pd.notna(row['販売価格_設定販売額']) else 0
                
                # デフォルト値設定
                fee_rate = None
                fee_category = "不明"
                media_fee = None  # メディア手数料の初期値
                
                # カテゴリ名に該当する情報がある場合
                if category_name in category_mapping:
                    category_info = category_mapping[category_name]
                    fee_category = category_info.get('販売手数料カテゴリ', "不明")
                    fee_rates = category_info.get('販売手数料率', [])
                    
                    # メディア手数料を取得し、あれば消費税(10%)を加算
                    base_media_fee = category_info.get('メディア手数料')
                    if base_media_fee is not None:
                        media_fee = base_media_fee * 1.1  # 消費税を加算
                        media_fee = round(media_fee)  # 四捨五入して整数に
                    
                    # 価格に応じた手数料率を決定
                    if isinstance(fee_rates, list):
                        # 配列形式の場合（新形式）
                        for rate_info in fee_rates:
                            upper_limit = rate_info.get('上限金額')
                            if upper_limit is None or price <= upper_limit:
                                fee_rate = rate_info.get('料率')
                                break
                    elif isinstance(fee_rates, dict):
                        # 辞書形式の場合（旧形式 - 互換性のため）
                        if price <= 750 and '750円以下' in fee_rates:
                            fee_rate = fee_rates['750円以下']
                        elif 750 < price <= 1500 and '750円超 1500円以下' in fee_rates:
                            fee_rate = fee_rates['750円超 1500円以下']
                        elif price > 1500 and '1500円超' in fee_rates:
                            fee_rate = fee_rates['1500円超']
                        elif '750円超' in fee_rates and price > 750:
                            fee_rate = fee_rates['750円超']
                        elif 'default' in fee_rates:
                            fee_rate = fee_rates['default']
                    else:
                        # 数値の場合（旧旧形式 - さらなる互換性のため）
                        fee_rate = fee_rates
                
                return pd.Series([category_name, fee_category, fee_rate, media_fee])
            
            # カテゴリ情報と手数料率を列に追加
            if 'カテゴリー' in result_df.columns and '販売価格_設定販売額' in result_df.columns:
                # apply関数で複数の値を同時に返す
                result_df[['商品情報_カテゴリ', '販売手数料カテゴリ', '手数料・利益_販売手数料率', '手数料・利益_メディア手数料']] = (
                    result_df.apply(get_category_info_and_fee_rate, axis=1)
                )
                
                # 手数料率をパーセント表示用に変換（例: 0.15 → 15%）
                result_df['手数料・利益_販売手数料率_表示用'] = result_df['手数料・利益_販売手数料率'].apply(
                    lambda x: f"{x*100:.1f}%" if pd.notna(x) else "対象外"
                )
                
                # 販売手数料の計算（最低販売手数料を考慮）
                def calculate_fee(row):
                    if pd.isna(row['手数料・利益_販売手数料率']) or pd.isna(row['販売価格_設定販売額']):
                        return None
                    
                    category_name = row['商品情報_カテゴリ']
                    min_fee = 0
                    if category_name in category_mapping:
                        min_fee = category_mapping[category_name].get('最低販売手数料', 0)
                    
                    calculated_fee = row['販売価格_設定販売額'] * row['手数料・利益_販売手数料率']
                    
                    # 最低手数料がnullの場合は最低料金の制約なし
                    if min_fee is None:
                        return calculated_fee
                    
                    # 最低手数料と計算手数料の大きい方を採用
                    return max(calculated_fee, min_fee)
                
                # 販売手数料を計算して列に追加（小数点第一位で四捨五入）
                result_df['手数料・利益_販売手数料'] = result_df.apply(calculate_fee, axis=1).apply(
                    lambda x: round(x) if pd.notna(x) else None
                )
                
                # 販売手数料（税込）を計算して列に追加（手数料に10%の消費税を加算し、小数点第一位で四捨五入）
                result_df['手数料・利益_販売手数料(税込)'] = result_df['手数料・利益_販売手数料'].apply(
                    lambda x: round(x * 1.1) if pd.notna(x) else None
                )
                
                logging.info("カテゴリ情報と販売手数料率、税込手数料、メディア手数料(税込)の列を追加しました")
            
            return result_df
            
        except Exception as e:
            logging.error(f"カテゴリ計算処理エラー: {str(e)}")
            traceback.print_exc()  # 詳細なエラー情報を出力
            return df


    def add_sourcing_price_calculations(self, df):
        """
        仕入れ情報（ネッシー、スーデリ、ヤフーショッピング）から
        最安値情報を計算するメソッド
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # 仕入れサイト情報の設定
            sourcing_sites = [
                {
                    'name': 'ネッシー',
                    'price_column': 'ネッシー_価格',
                    'is_tax_included': False,  # 税抜き価格の場合はFalse
                    'url_prefix': 'https://www.netsea.jp/search/?keyword=',
                    'url_column': None  # 特定のURL列がない場合はNone
                },
                {
                    'name': 'スーデリ',
                    'price_column': 'スーデリ_価格',
                    'is_tax_included': False,  # 税抜き価格の場合はFalse
                    'url_prefix': 'https://www.superdelivery.com/p/do/psl/?so=score&vi=1&sb=all&word=',
                    'url_column': None
                },
                {
                    'name': 'ヤフー',
                    'price_column': 'ヤフー_価格',
                    'is_tax_included': True,  # 税込み価格の場合はTrue
                    'url_prefix': '',  # URL列から直接取得するのでプレフィックスは不要
                    'url_column': 'ヤフー_商品URL'  # 特定のURL列がある場合はその列名
                }
                # 将来的に他の卸サイトを追加する場合は、ここに追加していく
            ]
            
            # 各行について最安値と対応するURLを計算
            def find_cheapest_price_and_url(row):
                min_price = float('inf')  # 初期値は無限大
                min_price_site = None
                
                for site in sourcing_sites:
                    price_column = site['price_column']
                    
                    # 列が存在し、値がある場合のみ処理
                    if price_column in row and pd.notna(row[price_column]):
                        # 価格を取得
                        price = float(row[price_column])
                        
                        # 税抜き価格の場合は税込みに変換
                        if not site['is_tax_included']:
                            price = price * 1.1
                        
                        # 最安値を更新
                        if price < min_price:
                            min_price = price
                            min_price_site = site
                
                # 最安値が見つからなかった場合
                if min_price_site is None:
                    return pd.Series([None, None])
                
                # URLの生成
                url = None
                if min_price_site['url_column'] and min_price_site['url_column'] in row:
                    # 特定のURL列がある場合はそこから取得
                    url = row[min_price_site['url_column']]
                else:
                    # URLプレフィックスとJANコードを結合
                    jan_code = row['JAN'] if pd.notna(row['JAN']) else ''
                    url = min_price_site['url_prefix'] + str(jan_code)
                
                return pd.Series([round(min_price), url])
            
            # 最安値とURLを列に追加
            if 'JAN' in result_df.columns:
                # 各サイトの価格列が存在するか確認
                existing_sites = []
                for site in sourcing_sites:
                    if site['price_column'] in result_df.columns:
                        existing_sites.append(site)
                
                if existing_sites:
                    print(f"📊 仕入れ価格計算: {len(existing_sites)}サイトの価格情報があります")
                    logging.info(f"仕入れ価格計算: {len(existing_sites)}サイトの価格情報があります")
                    
                    # サイト情報の表示
                    for site in existing_sites:
                        non_null_count = result_df[site['price_column']].notna().sum()
                        print(f"  - {site['name']}: {non_null_count}件の価格情報")
                    
                    # 最安値とURLを計算
                    result_df[['JAN価格_JAN価格下代(税込)', 'JAN価格_商品URL']] = result_df.apply(
                        find_cheapest_price_and_url, axis=1
                    )
                    
                    # 計算結果の統計
                    non_null_price = result_df['JAN価格_JAN価格下代(税込)'].notna().sum()
                    print(f"✅ JAN価格計算完了: {non_null_price}件の最安値情報を追加しました")
                    logging.info(f"JAN価格計算完了: {non_null_price}件の最安値情報を追加しました")
                else:
                    print("⚠️ 仕入れ価格列が見つかりません")
                    logging.warning("仕入れ価格列が見つかりません")
            else:
                print("⚠️ JAN列が見つからないため、仕入れ価格計算をスキップします")
                logging.warning("JAN列が見つからないため、仕入れ価格計算をスキップします")
            
            return result_df
            
        except Exception as e:
            logging.error(f"仕入れ価格計算処理エラー: {str(e)}")
            import traceback
            traceback.print_exc()  # 詳細なエラー情報を出力
            return df

    def add_yoriyasu_calculations(self, df):
        """
        ヨリヤスの情報を処理するメソッド
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # ヨリヤスの情報が存在するか確認
            yoriyasu_columns = [col for col in result_df.columns if col.startswith('ヨリヤス_')]
            if not yoriyasu_columns:
                print("⚠️ ヨリヤスの情報が見つかりません")
                logging.warning("ヨリヤスの情報が見つかりません")
                return result_df
            
            print(f"📊 ヨリヤス情報処理: {len(yoriyasu_columns)}列の情報があります")
            
            # 送料情報に基づいて価格表示を調整する関数
            def format_price_with_shipping(price, shipping):
                if pd.isna(price) or pd.isna(shipping):
                    return None
                    
                price_str = str(price)
                
                if shipping == '送料無料':
                    return price_str
                elif shipping == '条件付き送料無料':
                    return f"{price_str} (※)"
                elif shipping == '送料別':
                    return f"{price_str} (+)"
                else:
                    return price_str
            
            # 4つの仕入先情報を処理
            for i in range(1, 5):
                # 価格列
                price_col = f'ヨリヤス_仕入価格{i}'
                shipping_col = f'ヨリヤス_送料{i}'
                url_col = f'ヨリヤス_仕入れURL{i}'
                site_col = f'ヨリヤス_仕入れサイト{i}'
                
                # 対応する出力列
                output_price_col = f'ネット価格_仕入価格{i}'
                output_url_col = f'ネット価格_仕入れURL{i}'
                output_site_col = f'ネット価格_仕入れサイト{i}'
                
                # 価格情報の処理
                if price_col in result_df.columns and shipping_col in result_df.columns:
                    result_df[output_price_col] = result_df.apply(
                        lambda row: format_price_with_shipping(row[price_col], row[shipping_col]), 
                        axis=1
                    )
                    
                    # 値が入っている行数をカウント
                    non_null_count = result_df[output_price_col].notna().sum()
                    print(f"  - ネット価格{i}: {non_null_count}件の価格情報")
                
                # URL情報の処理
                if url_col in result_df.columns:
                    result_df[output_url_col] = result_df[url_col]
                
                # サイト情報の処理
                if site_col in result_df.columns:
                    result_df[output_site_col] = result_df[site_col]
            
            print(f"✅ ヨリヤス情報処理完了: 合計12列の情報を追加しました")
            return result_df
            
        except Exception as e:
            logging.error(f"ヨリヤス情報処理エラー: {str(e)}")
            import traceback
            traceback.print_exc()
            return df
        



In [ ]:
# セル5: 工程3（工程1と工程2の出力結果を使う)
class ProductCalculator(ProductCalculator):
    def add_profit_calculations(self, df):
        """
        手数料合計と利益に関する計算を行うメソッド
        各種手数料の合計や利益額、利益率を計算します
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # 手数料関連の列が存在するか確認
            fee_columns = [
                '手数料・利益_販売手数料(税込)',
                '手数料・利益_発送代行手数料',
                '手数料・利益_メディア手数料',
                '手数料・利益_月額保管料'
            ]
            
            # 存在する列のみを対象とする
            existing_fee_columns = [col for col in fee_columns if col in result_df.columns]
            
            if not existing_fee_columns:
                logging.warning("手数料関連の列が見つかりませんでした。手数料合計は計算できません。")
            else:
                # 手数料合計を計算する関数
                def calculate_total_fee(row):
                    total = 0
                    
                    # 各手数料を合計（Noneの場合は0として扱う）
                    for col in existing_fee_columns:
                        value = row[col]
                        if pd.notna(value):
                            total += value
                    
                    return total
                
                # 手数料合計を計算して列に追加
                result_df['手数料・利益_手数料合計'] = result_df.apply(calculate_total_fee, axis=1)
                logging.info("手数料合計の列を追加しました")
            
            # 利益額の計算
            if '販売価格_設定販売額' in result_df.columns and '手数料・利益_手数料合計' in result_df.columns:
                def calculate_profit(row):
                    # 販売価格を取得
                    selling_price = row['販売価格_設定販売額'] if pd.notna(row['販売価格_設定販売額']) else 0
                    
                    # 手数料合計を取得
                    total_fee = row['手数料・利益_手数料合計'] if pd.notna(row['手数料・利益_手数料合計']) else 0
                    
                    # 実質最安値（仕入れ価格）を取得
                    real_cost = 0
                    
                    # JAN価格_JAN価格下代(税込)を確認
                    jan_price = None
                    if 'JAN価格_JAN価格下代(税込)' in row and pd.notna(row['JAN価格_JAN価格下代(税込)']):
                        jan_price = float(row['JAN価格_JAN価格下代(税込)'])
                    
                    # ネット価格_仕入価格1を確認（末尾の(※)や(+)を除去して数値に変換）
                    net_price = None
                    if 'ネット価格_仕入価格1' in row and pd.notna(row['ネット価格_仕入価格1']):
                        price_str = str(row['ネット価格_仕入価格1'])
                        # 数値部分のみを抽出（(※)や(+)を除外）
                        import re
                        price_match = re.match(r'(\d+)', price_str)
                        if price_match:
                            net_price = float(price_match.group(1))
                    
                    # JAN価格とネット価格を比較して小さい方（安い方）を採用
                    if jan_price is not None and net_price is not None:
                        real_cost = min(jan_price, net_price)
                    elif jan_price is not None:
                        real_cost = jan_price
                    elif net_price is not None:
                        real_cost = net_price
                    
                    # 最安値を列に追加するためのヘルパー変数
                    row['販売価格_実質最安値'] = real_cost
                    
                    # 利益額を計算（販売価格 - 実質最安値 - 手数料合計）
                    profit = selling_price - real_cost - total_fee
                    
                    return round(profit)  # 小数点以下を四捨五入
                
                # 実質最安値の列を作成
                result_df['販売価格_実質最安値'] = 0
                
                # 利益額を計算して列に追加
                result_df['手数料・利益_利益額'] = result_df.apply(calculate_profit, axis=1)
                logging.info("利益額の列を追加しました")
                
                # 実質最安値が正しく設定されていることを確認
                # （calculate_profit関数内の変更は直接結果に反映されないため）
                result_df['販売価格_実質最安値'] = result_df.apply(
                    lambda row: self._calculate_real_cost(row), axis=1
                )
                logging.info("実質最安値の列を追加しました")
            
            # 利益率の計算
            if '手数料・利益_利益額' in result_df.columns and '販売価格_設定販売額' in result_df.columns:
                def calculate_profit_rate(row):
                    # 利益額を取得
                    profit = row['手数料・利益_利益額'] if pd.notna(row['手数料・利益_利益額']) else 0
                    
                    # 販売価格を取得
                    selling_price = row['販売価格_設定販売額'] if pd.notna(row['販売価格_設定販売額']) else 0
                    
                    # ゼロ除算を防ぐ
                    if selling_price == 0:
                        return None
                    
                    # 利益率を計算（利益額 ÷ 販売価格 × 100）
                    profit_rate = (profit / selling_price) * 100
                    
                    # 小数点第一位まで丸めて返す（例: 12.3%）
                    return round(profit_rate, 1)
                
                # 利益率を計算して列に追加
                result_df['手数料・利益_利益率'] = result_df.apply(calculate_profit_rate, axis=1)
                logging.info("利益率の列を追加しました")
            
            return result_df
            
        except Exception as e:
            logging.error(f"利益計算処理エラー: {str(e)}")
            import traceback
            traceback.print_exc()  # 詳細なエラー情報を出力
            return df
    
    def _calculate_real_cost(self, row):
        """
        実質最安値（仕入れ価格）を計算するヘルパーメソッド
        
        Parameters:
        -----------
        row : pandas.Series
            データ行
            
        Returns:
        --------
        float
            実質最安値
        """
        # JAN価格_JAN価格下代(税込)を確認
        jan_price = None
        if 'JAN価格_JAN価格下代(税込)' in row and pd.notna(row['JAN価格_JAN価格下代(税込)']):
            jan_price = float(row['JAN価格_JAN価格下代(税込)'])
        
        # ネット価格_仕入価格1を確認（末尾の(※)や(+)を除去して数値に変換）
        net_price = None
        if 'ネット価格_仕入価格1' in row and pd.notna(row['ネット価格_仕入価格1']):
            price_str = str(row['ネット価格_仕入価格1'])
            # 数値部分のみを抽出（(※)や(+)を除外）
            import re
            price_match = re.match(r'(\d+)', price_str)
            if price_match:
                net_price = float(price_match.group(1))
        
        # JAN価格とネット価格を比較して小さい方（安い方）を採用
        if jan_price is not None and net_price is not None:
            return min(jan_price, net_price)
        elif jan_price is not None:
            return jan_price
        elif net_price is not None:
            return net_price
        else:
            return 0

    def add_expected_sales_calculations(self, df):
        """
        期待販売数と期待利益に関する計算を行うメソッド
        期待販売数や期待利益を計算します
        
        Parameters:
        -----------
        df : pandas.DataFrame
            処理対象のデータフレーム
            
        Returns:
        --------
        pandas.DataFrame
            列が追加されたデータフレーム
        """
        try:
            # 元のデータフレームのコピーを作成
            result_df = df.copy()
            
            # 期待販売数(1ヶ月)の計算
            if '30日間_新品販売数' in result_df.columns and 'FBA数' in result_df.columns:
                def calculate_expected_sales(row):
                    # 月間販売数を取得
                    monthly_sales = row['30日間_新品販売数'] if pd.notna(row['30日間_新品販売数']) else 0
                    
                    # FBA出品者数を取得
                    fba_sellers = row['FBA数'] if pd.notna(row['FBA数']) else 0
                    
                    # 期待販売数を計算（月間販売数 ÷ (FBA出品者数 + 1)）
                    # ゼロ除算を防ぐため、分母が0の場合は0を返す
                    if fba_sellers + 1 == 0:
                        return 0
                    
                    expected_sales = monthly_sales / (fba_sellers + 1)
                    
                    # 整数に四捨五入
                    return round(expected_sales)
                
                # 期待販売数を計算して列に追加
                result_df['期待販売数・利益_販売期待数(1ヶ月)'] = result_df.apply(calculate_expected_sales, axis=1)
                logging.info("期待販売数(1ヶ月)の列を追加しました")
            else:
                # 必要な列がない場合はログに記録
                missing_columns = []
                if '30日間_新品販売数' not in result_df.columns:
                    missing_columns.append('30日間_新品販売数')
                if 'FBA数' not in result_df.columns:
                    missing_columns.append('FBA数')
                
                logging.warning(f"期待販売数の計算に必要な列が見つかりません: {', '.join(missing_columns)}")
                print(f"⚠️ 期待販売数の計算に必要な列がありません: {', '.join(missing_columns)}")
            
            # 期待利益(1ヶ月)の計算
            if '期待販売数・利益_販売期待数(1ヶ月)' in result_df.columns and '手数料・利益_利益額' in result_df.columns:
                def calculate_expected_profit_1month(row):
                    # 期待販売数を取得
                    expected_sales = row['期待販売数・利益_販売期待数(1ヶ月)'] if pd.notna(row['期待販売数・利益_販売期待数(1ヶ月)']) else 0
                    
                    # 利益額を取得
                    profit = row['手数料・利益_利益額'] if pd.notna(row['手数料・利益_利益額']) else 0
                    
                    # 期待利益を計算（期待販売数 × 利益額）
                    expected_profit = expected_sales * profit
                    
                    # 整数に四捨五入
                    return round(expected_profit)
                
                # 期待利益(1ヶ月)を計算して列に追加
                result_df['期待販売数・利益_期待利益(1ヶ月)'] = result_df.apply(calculate_expected_profit_1month, axis=1)
                logging.info("期待利益(1ヶ月)の列を追加しました")
            else:
                logging.warning("期待利益(1ヶ月)の計算に必要な列が見つかりません")
            
            # 期待利益(3ヶ月)の計算
            if '90日間_新品販売数' in result_df.columns and '手数料・利益_利益額' in result_df.columns:
                def calculate_expected_profit_3month(row):
                    # 3ヶ月の販売数を取得
                    sales_3month = row['90日間_新品販売数'] if pd.notna(row['90日間_新品販売数']) else 0
                    
                    # 期待販売数(3ヶ月)を計算
                    # 3ヶ月先は出品者数の増減が予測困難なため、平均的な出品者数として4を使用
                    # これは経験則に基づき、多くの商品が3ヶ月後に約4人の出品者に落ち着くという想定による
                    # (新雷神のサンプルから計算すると3ヶ月の期待利益の計算時の出品者数は4人 [3人＋自分] で固定していた)
                    expected_sales_3month = sales_3month / 4
                    
                    # 利益額を取得
                    profit = row['手数料・利益_利益額'] if pd.notna(row['手数料・利益_利益額']) else 0
                    
                    # 期待利益を計算（期待販売数(3ヶ月) × 利益額）
                    expected_profit = expected_sales_3month * profit
                    
                    # 整数に四捨五入
                    return round(expected_profit)
                
                # 期待利益(3ヶ月)を計算して列に追加
                result_df['期待販売数・利益_期待利益(3ヶ月)'] = result_df.apply(calculate_expected_profit_3month, axis=1)
                logging.info("期待利益(3ヶ月)の列を追加しました")
            else:
                missing_columns = []
                if '90日間_新品販売数' not in result_df.columns:
                    missing_columns.append('90日間_新品販売数')
                if '手数料・利益_利益額' not in result_df.columns:
                    missing_columns.append('手数料・利益_利益額')
                
                logging.warning(f"期待利益(3ヶ月)の計算に必要な列が見つかりません: {', '.join(missing_columns)}")
            
            return result_df
            
        except Exception as e:
            logging.error(f"期待販売数・利益計算処理エラー: {str(e)}")
            traceback.print_exc()  # 詳細なエラー情報を出力
            return df

In [ ]:
# セル6: メイン処理実行
class ProductCalculator(ProductCalculator):
    def process(self):
        """
        メイン処理を実行
        
        CSVファイルを読み込み、計算処理を行い、結果を保存します。
        """
        try:
            logging.info("計算処理を開始します")
            
            # データの読み込み
            df = self.load_data()
            
            # 列名の確認
            logging.info(f"入力データの列: {', '.join(df.columns)}")
            
            # 工程1: 基本的な計算処理
            result_df = self.add_calculation_columns(df)
            
            # 工程2-1: サイズ計算処理
            result_df = self.add_size_calculations(result_df)
    
            # 工程2-2: カテゴリ計算処理
            result_df = self.add_category_calculations(result_df)
            
            # 工程2-3-1: 仕入れ価格計算処理（ネッシー、スーデリ、ヤフー）
            result_df = self.add_sourcing_price_calculations(result_df)
            
            # 工程2-3-2: ヨリヤス情報処理
            result_df = self.add_yoriyasu_calculations(result_df)
            
            # 工程3-1: 手数料合計・利益計算処理
            result_df = self.add_profit_calculations(result_df)
            
            # 工程3-2: 期待販売数・期待利益計算処理
            result_df = self.add_expected_sales_calculations(result_df)
            
            # データの保存
            self.save_data(result_df)
            
            # 処理結果の概要を表示
            self.print_summary(df, result_df)
            
            logging.info("計算処理が正常に完了しました")
            return result_df
            
        except Exception as e:
            logging.error(f"処理全体でエラーが発生: {str(e)}")
            raise
    
    def print_summary(self, original_df, result_df):
        """
        処理結果の概要を表示
        
        Parameters:
        -----------
        original_df : pandas.DataFrame
            元のデータフレーム
        result_df : pandas.DataFrame
            処理後のデータフレーム
        """
        # 仕入れソース列を除外した表示用データフレームを作成
        display_df = result_df.copy()
        columns_to_drop = [col for col in display_df.columns if 
                         col.startswith('ネッシー_') or 
                         col.startswith('スーデリ_') or 
                         col.startswith('ヤフー_') or 
                         col.startswith('ヨリヤス_')]
        
        if columns_to_drop:
            display_df = display_df.drop(columns=columns_to_drop)
            print(f"ℹ️ 表示から{len(columns_to_drop)}列の仕入れソースデータを除外しました")
        
        # 追加された列（仕入れソース列を除く）
        new_columns = [col for col in display_df.columns if col not in original_df.columns]
        
        print("\n=== 処理結果のサマリー ===")
        print(f"・入力データ: {len(original_df)}行, {len(original_df.columns)}列")
        print(f"・出力データ: {len(display_df)}行, {len(display_df.columns)}列")
        print(f"・追加された列: {len(new_columns)}列")
        
        if new_columns:
            print("\n追加された列の一覧:")
            for col in new_columns:
                print(f"・{col}")
        
        print(f"\n✨ 処理が完了しました！")
        
        # 処理結果のサンプルとしてdisplay_dfを返す
        return display_df

In [ ]:
# セル7: 実行コード
if __name__ == "__main__":
    try:
        # 計算処理クラスのインスタンスを作成
        calculator = ProductCalculator()
        
        # 処理を実行
        result_df = calculator.process()
        
        # 処理結果の最初の数行を表示（print_summaryから返される表示用データフレームを使用）
        display_df = calculator.print_summary(calculator.load_data(), result_df)
        
        print("\n=== 処理結果のサンプル（最初の5行）===")
        pd.set_option('display.max_columns', None)
        pd.set_option('display.max_rows', 5)
        pd.set_option('display.width', None)
        display(display_df.head())
        
    except Exception as e:
        print(f"❌ エラーが発生しました: {str(e)}")
        import traceback
        traceback.print_exc()